<a href="https://colab.research.google.com/github/YoonJiHwan98/BigContest_Project/blob/Jihwan/code/Bigcontest_imblearn_rf_shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os

ROOT_DIR = "/content/drive/My Drive/빅콘테스트_2022/"
DATA_PATH = ROOT_DIR + "data/"
IMAGE_PATH = ROOT_DIR + "img/"

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
  path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
  print(path)
  print("그림 저장 : ", fig_id)
  if tight_layout:
    plt.tight_layout()
  plt.savefig(path, format=fig_extension, dpi=resolution)

def save_data(data, file_name, DATA_PATH=DATA_PATH):
  file_path = os.path.join(DATA_PATH, file_name)
  print(file_path)
  return data.to_csv(file_path, index=False)  

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/빅콘테스트_2022/data/new/train_prepared_log.csv')
label = pd.read_csv('/content/drive/MyDrive/빅콘테스트_2022/data/new/train_prepared_label_log.csv')
test = pd.read_csv('/content/drive/MyDrive/빅콘테스트_2022/data/new/target_prepared_log.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#train-validation split

seed = 42, Stratified Sampling 이용


In [ ]:

from sklearn.model_selection import StratifiedShuffleSplit

def split_data(data, label=label, test_size = 0.02, random_state = 42):
    split = StratifiedShuffleSplit(n_splits=5, test_size = test_size, random_state=random_state)
    
    for train_index, valid_index in split.split(data, label):
        train_data = data.loc[train_index]
        train_label = label.loc[train_index]
        valid_data = data.loc[valid_index]
        valid_label = label.loc[valid_index]

    return train_data, train_label, valid_data, valid_label
train_data, train_label, valid_data, valid_label = split_data(train, label)
train_data

In [ ]:
valid_data

# 모델 평가 함수

In [ ]:
SCORE_PATH = ROOT_DIR + "score/"
#IMAGE_PATH = 
os.makedirs(SCORE_PATH, exist_ok = True)
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# accuracy, precision, recall, f1 score
def model_validation(model, data, label, mode, file_title):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
  pred = model.predict(data)
  print('#### ' + mode + ' score ####')
  print(f'accuracy : {accuracy_score(label, pred)}, precision : {precision_score(label, pred)}, recall : {recall_score(label, pred)}, f1 score : {f1_score(label, pred)}')
  df = pd.DataFrame([], columns=['metric', 'score'])
  df.loc[0] = ['accuracy', accuracy_score(label, pred)]
  df.loc[1] = ['precision', precision_score(label, pred)]
  df.loc[2] = ['recall', recall_score(label, pred)]
  df.loc[3] = ['f1 score', f1_score(label, pred)]

  save_data(df, file_title, SCORE_PATH)
  return df
  
from sklearn import metrics
from sklearn.metrics import roc_auc_score


In [ ]:

# ROC curve, auc score
# proba  clf.predict_proba(X)[:, 1]

def model_auc(model, data, label, mode):

  proba = model.predict_proba(data)[::, 1]

  print('#### ' + mode + ' ROC AUC score ####')
  print(roc_auc_score(label, proba))

  print('#### ' + mode + ' ROC curve plotting####')
  fpr, tpr, _ = metrics.roc_curve(label, proba)

  plt.figure(figsize=(15,15))
  plt.plot(fpr, tpr)
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.title(mode + " ROC curve")
  save_fig(mode + " ROC curve")
  plt.show()

# imblearn_rf 모델 학습

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

bal_clf= BalancedRandomForestClassifier(random_state=42)

bal_clf.fit(train_data, train_label)  

In [ ]:
model_validation(bal_clf_tuned,valid_data, valid_label, 'RF_balanced_tuned','RF_balanced_tuned')

In [ ]:
from sklearn.model_selection import GridSearchCV

rf_tuned_parameters = [{ 'n_estimators' : [100],
           'min_samples_leaf' : [1, 2, 4, 8 ],
           'min_samples_split' : [2, 4, 8]
            }
]

rf_tuned = GridSearchCV(BalancedRandomForestClassifier(random_state=42),rf_tuned_parameters,cv = 2,scoring="f1",verbose=2,refit=True)
rf_tuned.fit(train_data,train_label)

In [ ]:
model_validation(rf_tuned,train_data, train_label, 'RF_balanced_tuned','RF_balanced_tuned_train.csv')

In [ ]:
model_validation(rf_tuned,valid_data, valid_label, 'RF_balanced_tuned','RF_balanced_tuned_valid.csv')

In [ ]:
rf_model.best_estimator_

In [ ]:
scores_df = pd.DataFrame(rf_tuned.cv_results_)
scores_df

# 모델 저장

In [ ]:
# 모델을 저장할 위치
MODEL_PATH = os.path.join(ROOT_DIR, "model/")
os.makedirs(MODEL_PATH, exist_ok = True)
#'/content/gdrive/My Drive/빅콘테스트_2022/model/'


In [ ]:

import pickle
pickle.dump(model, open(MODEL_PATH + 'lgb_shap_.pkl', 'wb'))

In [ ]:
rf_tuned.best_estimator_

In [ ]:
rf_tuned.cv_results_

# 모델 불러오기

In [ ]:
import pickle
MODEL_PATH = os.path.join(ROOT_DIR, "model/")

def import_model(model_name):
  with open(MODEL_PATH + model_name, 'rb') as file:
    model = pickle.load(file)
  return model
 

In [ ]:
with open('/content/drive/MyDrive/빅콘테스트_2022/model/balanced_rf_model_tuned.pkl', "rb") as file:
    rf_model = model = pickle.load(file)
    rf_model


# 모델 평가

In [ ]:
model_validation(rf_model,train_data, train_label, 'RF_balanced_tuned','RF_balanced_tuned_train.csv')

In [ ]:
model_validation(rf_model,valid_data, valid_label, 'RF_balanced_tuned','RF_balanced_tuned_valid.csv')

In [ ]:
rf_model.param_grid